In [1]:
%matplotlib inline

import warnings
import os
from pathlib import Path
import quandl
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz, _tree
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error, precision_recall_curve
from sklearn.preprocessing import Imputer
import statsmodels.api as sm
from scipy.interpolate import interp1d, interp2d

In [2]:
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

## Get Data

In [3]:
with pd.HDFStore('../data/assets.h5') as store:
    print(store.info())
    prices = store['quandl/wiki/prices'].adj_close.unstack('ticker')
    stocks = store['us_equities/stocks']

<class 'pandas.io.pytables.HDFStore'>
File path: ../data/assets.h5
/fred/assets                   frame        (shape->[4826,5])     
/quandl/wiki/prices            frame        (shape->[15389314,12])
/quandl/wiki/stocks            frame        (shape->[1,2])        
/sp500/prices                  frame        (shape->[37721,5])    
/sp500/stocks                  frame        (shape->[1,7])        
/us_equities/stocks            frame        (shape->[1,6])        


In [4]:
shared = prices.columns.intersection(stocks.index)
prices = prices.loc['2010': '2018', shared]
stocks = stocks.loc[shared, ['marketcap', 'ipoyear', 'sector']]

In [5]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2412 entries, A to ZUMZ
Data columns (total 3 columns):
marketcap    2408 non-null object
ipoyear      1065 non-null float64
sector       2372 non-null object
dtypes: float64(1), object(2)
memory usage: 75.4+ KB


In [6]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2113 entries, 2010-01-04 to 2018-03-27
Columns: 2412 entries, A to ZUMZ
dtypes: float64(2412)
memory usage: 38.9 MB


### Create monthly return series

Remove outliers

In [7]:
returns = prices.resample('M').pct_change().stack().swaplevel()
returns = returns[returns.between(left=returns.quantile(.05), right=returns.quantile(.95))].to_frame('returns')

### Lagged Returns

In [8]:
for t in range(1, 13):
    returns[f't-{t}'] = returns.groupby(level='ticker').returns.shift(t)
returns = returns.dropna()

### Time Period Dummies

In [9]:
# returns = returns.reset_index('date')
dates = returns.index.get_level_values('date')
returns['year'] = dates.year
returns['month'] = dates.month
returns = pd.get_dummies(returns, columns=['year', 'month'])

In [10]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 170515 entries, (A, 2011-02-28 00:00:00) to (ZUMZ, 2018-03-31 00:00:00)
Data columns (total 33 columns):
returns      170515 non-null float64
t-1          170515 non-null float64
t-2          170515 non-null float64
t-3          170515 non-null float64
t-4          170515 non-null float64
t-5          170515 non-null float64
t-6          170515 non-null float64
t-7          170515 non-null float64
t-8          170515 non-null float64
t-9          170515 non-null float64
t-10         170515 non-null float64
t-11         170515 non-null float64
t-12         170515 non-null float64
year_2011    170515 non-null uint8
year_2012    170515 non-null uint8
year_2013    170515 non-null uint8
year_2014    170515 non-null uint8
year_2015    170515 non-null uint8
year_2016    170515 non-null uint8
year_2017    170515 non-null uint8
year_2018    170515 non-null uint8
month_1      170515 non-null uint8
month_2      170515 non-null uint8
month_3      1

### Get stock characteristics

#### Create age proxy

In [11]:
stocks['age'] = pd.qcut(stocks.ipoyear, q=5, labels=list(range(1, 6))).astype(float).fillna(0).astype(int)
stocks = stocks.drop('ipoyear', axis=1)

#### Create size proxy

In [12]:
stocks.marketcap = stocks.marketcap.str.replace('$', '')
stocks['mcap'] = stocks.marketcap.str[-1]
stocks.marketcap =  pd.to_numeric(stocks.marketcap.str[:-1])
stocks = stocks[stocks.mcap.isin(['B', 'M'])]
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2407 entries, A to ZUMZ
Data columns (total 4 columns):
marketcap    2407 non-null float64
sector       2369 non-null object
age          2407 non-null int64
mcap         2407 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 94.0+ KB


In [13]:
stocks.marketcap = stocks.apply(lambda x: x.marketcap * 1000 if x.mcap == 'B' else x.marketcap, axis=1)
stocks['size'] = pd.qcut(stocks.marketcap, q=10, labels=list(range(1, 11)))
stocks = stocks.drop(['mcap', 'marketcap'], axis=1)

#### Create Dummy variables

In [14]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2407 entries, A to ZUMZ
Data columns (total 3 columns):
sector    2369 non-null object
age       2407 non-null int64
size      2407 non-null category
dtypes: category(1), int64(1), object(1)
memory usage: 59.2+ KB


In [15]:
stocks = pd.get_dummies(stocks, 
                        columns=['size', 'age',  'sector'], 
                        prefix=['size', 'age', ''], 
                        prefix_sep=['_', '_', ''])
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2407 entries, A to ZUMZ
Data columns (total 28 columns):
size_1                   2407 non-null uint8
size_2                   2407 non-null uint8
size_3                   2407 non-null uint8
size_4                   2407 non-null uint8
size_5                   2407 non-null uint8
size_6                   2407 non-null uint8
size_7                   2407 non-null uint8
size_8                   2407 non-null uint8
size_9                   2407 non-null uint8
size_10                  2407 non-null uint8
age_0                    2407 non-null uint8
age_1                    2407 non-null uint8
age_2                    2407 non-null uint8
age_3                    2407 non-null uint8
age_4                    2407 non-null uint8
age_5                    2407 non-null uint8
Basic Industries         2407 non-null uint8
Capital Goods            2407 non-null uint8
Consumer Durables        2407 non-null uint8
Consumer Non-Durables    2407 non-null uint

### Combine data

In [16]:
data = (returns
        .reset_index('date')
        .merge(stocks, left_index=True, right_index=True)
        .dropna()
        .set_index('date', append=True))

s = len(returns.columns)
data.iloc[:, s:] = data.iloc[:, s:].astype(int).apply(pd.to_numeric, downcast='integer')
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 170261 entries, (A, 2011-02-28 00:00:00) to (ZUMZ, 2018-03-31 00:00:00)
Data columns (total 61 columns):
returns                  170261 non-null float64
t-1                      170261 non-null float64
t-2                      170261 non-null float64
t-3                      170261 non-null float64
t-4                      170261 non-null float64
t-5                      170261 non-null float64
t-6                      170261 non-null float64
t-7                      170261 non-null float64
t-8                      170261 non-null float64
t-9                      170261 non-null float64
t-10                     170261 non-null float64
t-11                     170261 non-null float64
t-12                     170261 non-null float64
year_2011                170261 non-null uint8
year_2012                170261 non-null uint8
year_2013                170261 non-null uint8
year_2014                170261 non-null uint8
year_2015           

### Store data

In [17]:
with pd.HDFStore('data.h5') as store:
    store.put('data', data)